In [1]:
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML


#Writing File. Select top for Production. Bottom for Debug

# Prod - jupyter nbconvert --to html CompBal.ipynb --no-input --no-prompt
# Debug - jupyter nbconvert --to html CompBal.ipynb
# For no code PDF - jupyter-nbconvert CompBal.ipynb --no-input --no-prompt --to pdfviahtml

# MVP Competitive Balance Draft
***

The MVP Competitive Balance Draft Rounds A and B are designed to counteract imbalances in the league due simply to differences in the local market sizes of our teams. The 10 teams with the smallest markets (as determined by local media revenue from the previous season) will be eligible for these picks. The order of these picks will be determined by a lottery drawing with each team receiving a certain number of balls based on three components: Local Media Revenue and All Other Revenue. There will be 10 picks awarded. 6 at the end of the first round, and 4 at the end of the 2nd round. The rest of this document outlines this process in detail, and shows the results at each step.

In [2]:
#Season

season = 2027
league_id = 153

In [3]:
index = [1,2,3,4,5,6,7,8,9,10]
balls_base = pd.DataFrame([250,200,150,100,80,70,60,40,30,20], columns = ['Balls'], index = index)
balls_base

,Balls
1,250
2,200
3,150
4,100
5,80
6,70
7,60
8,40
9,30
10,20


Balls will be assigned according to the above weights twice using the following two components:

1. **Market Size in Millions.** This is the team's Media Revenue during the prior season. The bottom 10 Media Revenues will be granted eligibility into the competitive balance pool. Balls are assigned by lowest Media Revenue to highest among these bottom 10 teams.

    > Tiebreakers are handled by giving the average number of available chances for selection to all tied teams. For example, in this season, we have a tie for 2nd, 3rd, and 4th. The total number of lottery chances for those positions are 200, 150, 100 respectively. Given the sum of those chances is 450, each team receives 150 chances in the lottery draw.

    > Playoff teams are eligible for Competitive Balance picks. However, they will not be a part of the lottery and instead will pick after the other picks have been assigned. The order of picks between playoff teams will be determined by team record in the prior season (low to high).

    
2. **Other Revenue Percentage.** This represents the % of Revenue a team earns outside of Media Revenue. The balls for this component will be assigned in descending order to prevent teams from tanking, doing nothing to earn incomes, and simply to incentivize even small market teams to take initiatives to sell tickets and bring in funds as best they can, which is in everyone's best interest. Only the same 10 teams from Step 1 are eligible for these balls.


Below is a list of all teams, sorted alphabetically with all criteria for selection. 

In [5]:
path = 'C:/Users/night/OneDrive/Documents/Out of the Park Developments/OOTP Baseball 22/saved_games/MVP.lg/import_export/csv/'

teams = pd.read_csv(path + 'teams.csv')
team_history = pd.read_csv(path + 'team_history.csv')
team_financials = pd.read_csv(path + 'team_history_financials.csv')
standings = pd.read_csv(path + 'team_history_record.csv')


team_history = team_history[(team_history['year'] == season) & (team_history['league_id'] == league_id)][['team_id', 'made_playoffs']]
team_financials = team_financials[(team_financials['year'] == season) & (team_financials['league_id'] == league_id)]
standings = standings[(standings['year'] == season) & (standings['league_id'] == league_id)][['team_id', 'w']]

standings = standings.merge(teams, how = 'left',
                            left_on = ['team_id'],
                           right_on = ['team_id'])

standings['Team'] = standings['name'] + ' ' + standings['nickname']
standings = standings[['Team', 'w']]

comp_balance = team_financials.merge(teams, how = 'left', 
                      left_on = ['team_id','sub_league_id','division_id'], 
                      right_on = ['team_id','sub_league_id','division_id']
                     ).merge(team_history, how = 'left', 
                      left_on = ['team_id'], 
                      right_on = ['team_id'])

comp_balance['Team'] = comp_balance['name'] + ' ' + comp_balance['nickname']
comp_balance['Total Revenue'] = comp_balance['gate_revenue'] + \
                          comp_balance['season_ticket_revenue'] + \
                          comp_balance['media_revenue'] + \
                          comp_balance['merchandising_revenue'] + \
                          comp_balance['playoff_revenue']
comp_balance['Market Size'] = comp_balance['media_revenue']
comp_balance['Market Size in Millions'] = comp_balance['Market Size'] / 1000000
comp_balance['Playoff Team'] = np.where(comp_balance['made_playoffs'] == 1, 'Y','N')
comp_balance['Media Revenue Percentage'] = round((comp_balance['media_revenue'] / comp_balance['Total Revenue'])*100, 1)
comp_balance['Other Revenue Percentage'] = round(((comp_balance['Total Revenue'] - comp_balance['media_revenue']) / comp_balance['Total Revenue'])*100, 1)
comp_balance = comp_balance[['Team','Market Size in Millions', 'Media Revenue Percentage', 'Other Revenue Percentage', 'Playoff Team']]

comp_balance = comp_balance.sort_values('Team')
comp_balance.index = np.arange(1,len(comp_balance)+1)
comp_balance

,Team,Market Size in Millions,Media Revenue Percentage,Other Revenue Percentage,Playoff Team
1,Arizona Diamondbacks,76.0,51.7,48.3,N
2,Atlanta Braves,100.0,48.6,51.4,N
3,Baltimore Orioles,72.0,47.9,52.1,N
4,Boston Red Sox,120.0,38.3,61.7,Y
5,Chicago Cubs,115.0,42.0,58.0,Y
6,Chicago White Sox,85.0,41.4,58.6,N
7,Cincinnati Reds,92.5,37.0,63.0,Y
8,Cleveland Indians,87.5,37.2,62.8,Y
9,Colorado Rockies,100.0,52.8,47.2,N
10,Detroit Tigers,77.0,54.1,45.9,N


## Market size in millions Eligibiity

Below are the lowest 10 teams, sorted by Market Size in Millions. These 10 teams are eligible for a competitive balance draft pick in the upcoming draft.

The following ties exist in this season's comp balance eligibility: 

- 1st Place -> 2nd Place. 

The total balls available for 1st Place -> 2nd Place is 450. Each team in that group will recieve 225 balls into the lottery.

Finally, playoff teams will recieve 0 lottery balls by default. This will not increase the allocation for other teams.

In [6]:
market_10 = comp_balance.sort_values(by = 'Market Size in Millions').head(10).reset_index(drop = True)
market_10

,Team,Market Size in Millions,Media Revenue Percentage,Other Revenue Percentage,Playoff Team
0,Pittsburgh Pirates,70.0,59.4,40.6,N
1,Kansas City Royals,70.0,51.4,48.6,N
2,Miami Marlins,71.0,47.5,52.5,N
3,Baltimore Orioles,72.0,47.9,52.1,N
4,Tampa Bay Rays,74.0,34.2,65.8,Y
5,Arizona Diamondbacks,76.0,51.7,48.3,N
6,Detroit Tigers,77.0,54.1,45.9,N
7,Seattle Mariners,78.0,51.6,48.4,N
8,Chicago White Sox,85.0,41.4,58.6,N
9,Toronto Blue Jays,85.0,61.1,38.9,N


In [7]:
balls = balls_base.copy(deep = True)
balls.loc[1,'Market Balls'] = (balls_base.loc[1,'Balls'] + balls_base.loc[2,'Balls']) / 2
balls.loc[2,'Market Balls'] = (balls_base.loc[1,'Balls'] + balls_base.loc[2,'Balls']) / 2
balls.loc[3,'Market Balls'] = balls_base.loc[3,'Balls']
balls.loc[4,'Market Balls'] = balls_base.loc[4,'Balls']
balls.loc[5,'Market Balls'] = balls_base.loc[5,'Balls']
balls.loc[6,'Market Balls'] = balls_base.loc[6,'Balls']
balls.loc[7,'Market Balls'] = balls_base.loc[7,'Balls']
balls.loc[8,'Market Balls'] = balls_base.loc[8,'Balls']
balls.loc[9,'Market Balls'] = (balls_base.loc[9,'Balls'] + balls_base.loc[10,'Balls']) / 2
balls.loc[10,'Market Balls'] = (balls_base.loc[9,'Balls'] + balls_base.loc[10,'Balls']) / 2
balls = pd.DataFrame(balls['Market Balls']).astype(int)
balls

,Market Balls
1,225
2,225
3,150
4,100
5,80
6,70
7,60
8,40
9,25
10,25


In [8]:
market_10.index = np.arange(1, len(market_10) + 1)
market = pd.concat([market_10,balls], axis = 1)
market['Market Balls'] = np.where(market['Playoff Team'] == 'Y', 0, market['Market Balls'])
market[['Team','Market Size in Millions','Media Revenue Percentage','Playoff Team','Market Balls']]

,Team,Market Size in Millions,Media Revenue Percentage,Playoff Team,Market Balls
1,Pittsburgh Pirates,70.0,59.4,N,225
2,Kansas City Royals,70.0,51.4,N,225
3,Miami Marlins,71.0,47.5,N,150
4,Baltimore Orioles,72.0,47.9,N,100
5,Tampa Bay Rays,74.0,34.2,Y,0
6,Arizona Diamondbacks,76.0,51.7,N,70
7,Detroit Tigers,77.0,54.1,N,60
8,Seattle Mariners,78.0,51.6,N,40
9,Chicago White Sox,85.0,41.4,N,25
10,Toronto Blue Jays,85.0,61.1,N,25


## Other Revenue Eligibility

Below are the eligible teams sorted by Other Revenue Percentage, descending. Given no ties exist, the lottery balls are assigned sequentially. As noted before, playoff teams will recieve no lottery balls for the drawing.

In [9]:
other_rev = market.sort_values('Other Revenue Percentage', ascending = False)
other_rev.index = np.arange(1,len(other_rev) +1)
other_rev = other_rev.join(balls_base)
other_rev['Other Rev Balls'] = np.where(other_rev['Playoff Team'] == 'Y', 0, other_rev['Balls'])
other_rev[['Team', 'Other Revenue Percentage', 'Playoff Team', 'Other Rev Balls']]


,Team,Other Revenue Percentage,Playoff Team,Other Rev Balls
1,Tampa Bay Rays,65.8,Y,0
2,Chicago White Sox,58.6,N,200
3,Miami Marlins,52.5,N,150
4,Baltimore Orioles,52.1,N,100
5,Kansas City Royals,48.6,N,80
6,Seattle Mariners,48.4,N,70
7,Arizona Diamondbacks,48.3,N,60
8,Detroit Tigers,45.9,N,40
9,Pittsburgh Pirates,40.6,N,30
10,Toronto Blue Jays,38.9,N,20


## Final Draft Lottery Probabilities

For the Comp Balance Lottery, here are the chances provided for each team for the top selection. There's an important distinction to advise you on. This lottery is drawn randomly, without replacement, with a caveat. The process works as follows: 

1. The program draws a ball. The team on the ball is awarded the selection.
2. All corresponding balls are removed from the lottery pool.
3. The program then draws another ball. This only includes the remaining teams that have lottery balls.
4. Once the playoff teams come up, they are ranked from lowest win total, to highest.

This process repeats until the draft is over. So the probabilities you see below are exclusive to the first overall selection. Upon request, I can provide probabilities for each subsequent round if you are interested in seeing it. 

The draft lottery itself is not displayed below, simply the results. The code is provided upon request. 
> Side note: for those interested in the programatic methodology: The process is done through a manual iteration of 10 steps (10 picks) by utilizing the random package in python. The random_choices() method allows us to provide a list of teams, with assigned weights by constructing vectorized operations on those columns in the dataset. From there, we iterate 1 output of random choice. I then reconstruct the dataset to exclude the team selected so as to properly remove them from the process. This process is then repeated until the draft is over. 

In [10]:
final = other_rev[['Team', 'Market Size in Millions','Media Revenue Percentage','Other Revenue Percentage', 'Playoff Team', 'Market Balls', 'Other Rev Balls']]
final['Total Balls'] = final['Market Balls'] + final['Other Rev Balls']
final['% Pick Chance'] = round((final['Total Balls']/sum(final['Total Balls']))*100,1)

final = final.merge(standings, left_on = "Team", right_on = 'Team')
final = final.rename(columns = {'w': "Wins"})


final_cols = []
final = final[['Team', 
               'Wins',
               'Market Size in Millions', 
               'Media Revenue Percentage', 
               'Other Revenue Percentage', 
               'Total Balls',
               '% Pick Chance']].sort_values(['Total Balls', 'Wins'], ascending = (False,True)).set_index('Team')

#final[['Market Size in Millions', 'Media Revenue Percentage', 'Other Revenue Percentage', 'Total Balls','% Pick Chance']]
final

,Wins,Market Size in Millions,Media Revenue Percentage,Other Revenue Percentage,Total Balls,% Pick Chance
Team,,,,,,
Kansas City Royals,74,70.0,51.4,48.6,305,18.3
Miami Marlins,86,71.0,47.5,52.5,300,18.0
Pittsburgh Pirates,74,70.0,59.4,40.6,255,15.3
Chicago White Sox,81,85.0,41.4,58.6,225,13.5
Baltimore Orioles,90,72.0,47.9,52.1,200,12.0
Arizona Diamondbacks,81,76.0,51.7,48.3,130,7.8
Seattle Mariners,76,78.0,51.6,48.4,110,6.6
Detroit Tigers,59,77.0,54.1,45.9,100,6.0
Toronto Blue Jays,68,85.0,61.1,38.9,45,2.7


## Final Results

The following picks will be awarded as follows:

In [11]:
picks = {}

random.seed(10182021)

first = random.choices(final.index, final['Total Balls'], k = 1)
two = final[final.index != first[0]]
two['% Pick Chance'] = round(two['Total Balls']/sum(two['Total Balls'])*100,1)
picks['Round 1 - Pick C1'] = [first[0],final.loc[final.index == first[0]]['% Pick Chance'][0], final.loc[final.index == first[0]]['Total Balls'][0], sum(final['Total Balls']) ]


second = random.choices(two.index, two['Total Balls'], k = 1)
three = two[two.index != second[0]]
three['% Pick Chance'] = round(three['Total Balls']/sum(three['Total Balls'])*100,1)
picks['Round 1 - Pick C2'] = [second[0],two.loc[two.index == second[0]]['% Pick Chance'][0], two.loc[two.index == second[0]]['Total Balls'][0], sum(two['Total Balls']) ]

third = random.choices(three.index, three['Total Balls'], k = 1)
four = three[three.index != third[0]]
four['% Pick Chance'] = round(four['Total Balls']/sum(four['Total Balls'])*100,1)
picks['Round 1 - Pick C3'] = [third[0],three.loc[three.index == third[0]]['% Pick Chance'][0], three.loc[three.index == third[0]]['Total Balls'][0], sum(three['Total Balls']) ]

fourth = random.choices(four.index, four['Total Balls'], k = 1)
five = four[four.index != fourth[0]]
five['% Pick Chance'] = round(five['Total Balls']/sum(five['Total Balls'])*100,1)
picks['Round 1 - Pick C4'] = [fourth[0],four.loc[four.index == fourth[0]]['% Pick Chance'][0], four.loc[four.index == fourth[0]]['Total Balls'][0], sum(four['Total Balls']) ]

fifth = random.choices(five.index, five['Total Balls'], k = 1)
six = five[five.index != fifth[0]]
six['% Pick Chance'] = round(six['Total Balls']/sum(six['Total Balls'])*100,1)
picks['Round 1 - Pick C5'] = [fifth[0],five.loc[five.index == fifth[0]]['% Pick Chance'][0], five.loc[five.index == fifth[0]]['Total Balls'][0], sum(five['Total Balls']) ]

sixth = random.choices(six.index, six['Total Balls'], k = 1)
seven = six[six.index != sixth[0]]
seven['% Pick Chance'] = round(seven['Total Balls']/sum(seven['Total Balls'])*100,1)
picks['Round 1 - Pick C6'] = [sixth[0],six.loc[six.index == sixth[0]]['% Pick Chance'][0], six.loc[six.index == sixth[0]]['Total Balls'][0], sum(six['Total Balls']) ]

seventh = random.choices(seven.index, seven['Total Balls'], k = 1)
eight = seven[seven.index != seventh[0]].sort_values('Wins', ascending = True)
eight['% Pick Chance'] = 100
eight['% Pick Chance'] = round(eight['Total Balls']/sum(eight['Total Balls'])*100,1)
picks['Round 2 - Pick C1'] = [seventh[0],seven.loc[seven.index == seventh[0]]['% Pick Chance'][0], seven.loc[seven.index == seventh[0]]['Total Balls'][0], sum(seven['Total Balls']) ]

#eighth = random.choices(eight.index, eight['Total Balls'], k = 1)
eighth = eight[eight['Wins'] == min(eight['Wins'])].index.tolist()
nine = eight[eight.index != eighth[0]]
nine['% Pick Chance'] = 100
#nine['% Pick Chance'] = round(nine['Total Balls']/sum(nine['Total Balls'])*100,1).fillna(100)
picks['Round 2 - Pick C2'] = [eighth[0],100, 0, 0]
#picks['Round 2 - Pick C2'] = final.reset_index().iloc[9,0]

#ninth = random.choices(nine.index, nine['Total Balls'], k = 1)
ninth = nine[nine['Wins'] == min(nine['Wins'])].index.tolist()
ten = nine[nine.index != ninth[0]]
ten['% Pick Chance'] = 100
#ten['% Pick Chance'] = round(ten['Total Balls']/sum(ten['Total Balls'])*100,1).fillna(100)
picks['Round 2 - Pick C3'] = [ninth[0],100, 0, 0]
#picks['Round 2 - Pick C3'] = ninth

#tenth = random.choices(ten.index, ten['Total Balls'], k = 1)
tenth = ten[ten['Wins'] == min(ten['Wins'])].index.tolist()
picks['Round 2 - Pick C4'] = [tenth[0],100, 0, 0]
#picks['Round 2 - Pick C4'] = tenth

for key, value in picks.items():
    print(str(key) +" : " + str(value[0]) + ' - Chance to win pick = ' + str(value[1]) + ' (' + str(value[2]) + '/' + str(value[3]) + ')')

Round 1 - Pick C1 : Pittsburgh Pirates - Chance to win pick = 15.3 (255/1670)
Round 1 - Pick C2 : Seattle Mariners - Chance to win pick = 7.8 (110/1415)
Round 1 - Pick C3 : Arizona Diamondbacks - Chance to win pick = 10.0 (130/1305)
Round 1 - Pick C4 : Chicago White Sox - Chance to win pick = 19.1 (225/1175)
Round 1 - Pick C5 : Baltimore Orioles - Chance to win pick = 21.1 (200/950)
Round 1 - Pick C6 : Kansas City Royals - Chance to win pick = 40.7 (305/750)
Round 2 - Pick C1 : Toronto Blue Jays - Chance to win pick = 10.1 (45/445)
Round 2 - Pick C2 : Detroit Tigers - Chance to win pick = 100 (0/0)
Round 2 - Pick C3 : Miami Marlins - Chance to win pick = 100 (0/0)
Round 2 - Pick C4 : Tampa Bay Rays - Chance to win pick = 100 (0/0)
